In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import re
import pycountry
import openpyxl
import random
import numpy as np

df = pd.read_json('../../data/tw/prokanon.tw_combs4.json')
df.loc[(df['count_max'] == 1000), 'count_max'] = df['count_min']
df['count_max'] = df['count_max'].apply(lambda x: x if x >= 961 else random.randint(1, 961))

df = df.loc[~(df['age_range'] == 'AGE_18_TO_24')]
df = df.loc[~(df['age_range'] == 'AGE_18_TO_34')]
df = df.loc[~(df['age_range'] == 'AGE_21_TO_34')]
df = df.loc[~(df['age_range'] == 'AGE_35_TO_49')]


df.sort_values(by='count_max',inplace=True)


df['cumsum'] = df.groupby('country')['count_max'].cumsum()

sum_per_country = df.groupby('country')['count_max'].sum()

df['cdf'] = df['cumsum'] / df['country'].map(sum_per_country)


paises = df[['country']].drop_duplicates()

In [2]:
df['count_max'].value_counts().sort_index()

count_max
2           5
3           3
4           3
5           2
6           3
           ..
11462880    1
13389263    1
13479745    1
15391739    1
21696894    1
Name: count, Length: 2882, dtype: int64

In [3]:

res_geq090 = df[df['cdf'] >= 0.1]

idx_min = res_geq090.groupby('country')['cdf'].apply(lambda x: (x - 0.1).abs().idxmin())

results_90 = res_geq090.loc[idx_min]



res_geq095 = df[df['cdf'] >= 0.05]

idx_min = res_geq095.groupby('country')['cdf'].apply(lambda x: (x - 0.05).abs().idxmin())

results_95 = res_geq095.loc[idx_min]



res_geq099 = df[df['cdf'] >= 0.01]

idx_min = res_geq099.groupby('country')['cdf'].apply(lambda x: (x - 0.01).abs().idxmin())

results_99 = res_geq099.loc[idx_min]




for index, row in results_90[['country','count_max']].iterrows():
    mask = paises['country'] == row['country']
    paises.loc[mask,'K^4_90'] = row['count_max']


for index, row in results_95[['country','count_max']].iterrows():
    mask = paises['country'] == row['country']
    paises.loc[mask,'K^4_95'] = row['count_max']


for index, row in results_99[['country','count_max']].iterrows():
    mask = paises['country'] == row['country']
    paises.loc[mask,'K^4_99'] = row['count_max']



for index, value in sum_per_country.items():
    mask = paises['country'] == index
    paises.loc[mask,'RK^4_90'] = (paises.loc[mask,'K^4_90']/value) * 100 
    paises.loc[mask,'RK^4_95'] = (paises.loc[mask,'K^4_95']/value) * 100 
    paises.loc[mask,'RK^4_99'] = (paises.loc[mask,'K^4_99']/value) * 100 
print(len(df.index))

5112


In [4]:
paises

,country,K^4_90,K^4_95,K^4_99,RK^4_90,RK^4_95,RK^4_99
4794,LK,11530.0,5154.0,1424.0,3.496822,1.563107,0.431871
11812,DJ,594.0,394.0,231.0,2.141621,1.420536,0.832853
8182,NZ,22625.0,14506.0,9840.0,2.525450,1.619190,1.098361
1900,LT,23061.0,19766.0,4378.0,2.772351,2.376232,0.526315
11479,SO,2922.0,1785.0,343.0,2.238223,1.367292,0.262735
...,...,...,...,...,...,...,...
846,HR,20680.0,13847.0,3007.0,3.013830,2.018013,0.438230
1178,FR,576567.0,294835.0,138262.0,3.413244,1.745406,0.818503
58,CA,379535.0,240912.0,111651.0,3.056724,1.940273,0.899222
4030,JP,1947139.0,1499673.0,855175.0,2.431416,1.872660,1.067867


In [5]:
paises.dropna(inplace=True)
paises = paises.astype({'K^4_90': int,'K^4_95': int,'K^4_99':int})
paises

,country,K^4_90,K^4_95,K^4_99,RK^4_90,RK^4_95,RK^4_99
4794,LK,11530,5154,1424,3.496822,1.563107,0.431871
11812,DJ,594,394,231,2.141621,1.420536,0.832853
8182,NZ,22625,14506,9840,2.525450,1.619190,1.098361
1900,LT,23061,19766,4378,2.772351,2.376232,0.526315
11479,SO,2922,1785,343,2.238223,1.367292,0.262735
...,...,...,...,...,...,...,...
846,HR,20680,13847,3007,3.013830,2.018013,0.438230
1178,FR,576567,294835,138262,3.413244,1.745406,0.818503
58,CA,379535,240912,111651,3.056724,1.940273,0.899222
4030,JP,1947139,1499673,855175,2.431416,1.872660,1.067867


In [6]:
nan_rows = paises[paises.isna().any(axis=1)]
print(nan_rows)

Empty DataFrame
Columns: [country, K^4_90, K^4_95, K^4_99, RK^4_90, RK^4_95, RK^4_99]
Index: []


In [7]:
paises.dropna(inplace=True)
paises['K^4_90'] = paises['K^4_90'].astype('int')
paises['K^4_95'] = paises['K^4_95'].astype('int')
paises['K^4_99'] = paises['K^4_99'].astype('int')
paises['RK^4_90'] = paises['RK^4_90'].round(3)
paises['RK^4_95'] = paises['RK^4_95'].round(3)
paises['RK^4_99'] = paises['RK^4_99'].round(3)



paises.to_parquet('../../data/analyzed/tw/TW_K^4_X_by_country.parquet')
paises

,country,K^4_90,K^4_95,K^4_99,RK^4_90,RK^4_95,RK^4_99
4794,LK,11530,5154,1424,3.497,1.563,0.432
11812,DJ,594,394,231,2.142,1.421,0.833
8182,NZ,22625,14506,9840,2.525,1.619,1.098
1900,LT,23061,19766,4378,2.772,2.376,0.526
11479,SO,2922,1785,343,2.238,1.367,0.263
...,...,...,...,...,...,...,...
846,HR,20680,13847,3007,3.014,2.018,0.438
1178,FR,576567,294835,138262,3.413,1.745,0.819
58,CA,379535,240912,111651,3.057,1.940,0.899
4030,JP,1947139,1499673,855175,2.431,1.873,1.068
